In [9]:
from flask import Flask, render_template, redirect, url_for, request,session
from flask import session
import pymongo
from pymongo import MongoClient
client = MongoClient("localhost",27017)
client = pymongo.MongoClient()
mydb = client.chirag
user_table=mydb.users_uq
post_table=mydb.posts
follow_table=mydb.follow
app = Flask(__name__)
app.secret_key = "test"
@app.route('/', methods=['GET', 'POST'])
def login():
    if 'user' not in session:
        session['user']="guest"
    if request.method =='GET':
        if session['user']!="guest":
            return redirect("/home")     
        else:
            return render_template("login.html")
    if request.method =='POST':
        username=request.form['username'] 
        password=request.form['password']
        print(username)
        user={"username":username}
        user=list(user_table.find(user))
        if len(user)>0:
            if(username==user[0]['username'] and password==user[0]['password']):
                error = 'Success Loggedin'
                session['user']=username
                return redirect('/home')                
            else:
                return render_template('login.html',resp="Error Username/Password invalid")  
@app.route('/home', methods=['GET', 'POST'])
def home():    
    if request.method == "GET":
        print(session['user'])
        if session['user']=="guest":
            return render_template('home.html',error="Please login to view posts")
        else:
            username=session['user']
            items=[]
            follows=list(follow_table.find({"username": username}))
            populate=[]
            for i in follows:
                username=i['follows']
                postsm=list(post_table.find({"username": username})) 
                if len(postsm)>0:        
                    for i in postsm:            
                        items.append({'post':i['post'],'user':i['username']})   
            username = session['user'] 
            return render_template('home.html', error="HOME",username=username,items=items)
    if request.method == "POST":
        username=session['user']
        if('posting' in request.form and request.form['short_post']!=""):
            post_v=request.form['short_post']
            post_table.insert({'username':username,"post":post_v})     
            follows=list(follow_table.find({"username": username}))
            populate=[]
            items=[]
            for i in follows:
                username=i['follows']            
                postsm=list(post_table.find({"username": username})) 
                if len(postsm)>0:        
                    for i in postsm:            
                        items.append({'post':i['post'],'user':i['username']})   
            username = session['user'] 
            return render_template('home.html', error="HOME",username=username,items=items)
        if 'delete' in request.form:
            temp=request.form['delete']
            post_table.delete_one({"post":temp})
            follows=list(follow_table.find({"username": username}))
            populate=[]
            items=[]
            for i in follows:
                username=i['follows']            
                postsm=list(post_table.find({"username": username})) 
                if len(postsm)>0:        
                    for i in postsm:            
                        items.append({'post':i['post'],'user':i['username']})   
            username = session['user'] 
            return render_template('home.html', error="HOME",username=username,items=items)
        else:
            return redirect("/logout")
@app.route('/user/<username1>', methods=['GET', 'POST'])
def user(username1): 
    if 'user' in session:
        if session['user']=="guest":
            return render_template('user.html', errorm="Please login to view posts",username=username1)
        
    if username1==session['user']:        
        if request.method == "GET":
            username = session['user']    
            items=[]
            follows=list(follow_table.find({"username": username}))
            populate=[]
            for i in follows:
                username=i['follows']
                postsm=list(post_table.find({"username":username})) 
                if len(postsm)>0:        
                    for i in postsm:            
                        items.append({'post':i['post'],'user':i['username']})  
            username = session['user'] 
            return render_template('user.html', error="HOME",username=username,items=items,scheck="selfaccount",cur=username)
        if request.method == "POST":
            username = session['user']
            print(request.form)
            if('post' in request.form):
                post_v=request.form['short_post']
                post_table.insert({'username':username,"post":post_v}) 
                username = session['user']    
                items=[]
                follows=list(follow_table.find({"username": username}))
                populate=[]
                for i in follows:
                    username=i['follows']
                    postsm=list(post_table.find({"username":username})) 
                    if len(postsm)>0:        
                        for i in postsm:            
                            items.append({'post':i['post'],'user':i['username']})  
                username = session['user'] 
                return render_template('user.html', error="HOME",username=username,items=items,scheck="selfaccount",cur=username)
            if(request.form['logout1']=="logout"):
                return redirect("/logout")
    else:
        username = session['user']    
        profile_watched = username1
        items=[]
        postsm=list(post_table.find({"username":profile_watched})) 
        if len(postsm)>0:        
            for i in postsm:            
                items.append({'post':i['post'],'user':i['username']})
        stat="Follow"
        status=list(follow_table.find({"username":username}))
        for i in status:
            if(i['follows']==profile_watched):
                stat="Following"    
                break      
        if request.method =="GET":
            username = session['user'] 
            return render_template('user.html', error="HOME",username=username1,items=items,stat=stat,cur=username)
        if request.method == "POST":
            if("follow" in request.form):
                follow_table.insert({"username":username,"follows":username1})
                return render_template('user.html', error="HOME",username=username1,items=items,self="false",stat="Following",cur=username)
            if("unfollow" in request.form):
                follow_table.delete_one({"username":username,"follows":username1})
                return render_template('user.html', error="HOME",username=username1,items=items,self="false",stat="Follow",cur=username)
            if(request.form['logout']=="logout"):
                return redirect("/logout")
@app.route('/logout', methods=['GET', 'POST'])
def logout():    
    session['user']="guest"
    return redirect("/")
@app.route('/signup', methods=['GET', 'POST'])
def signup():
    if request.method=="GET":
        return render_template("reg.html")
    else:
        if(request.form['username']!=""):
            username=request.form['username']
            if(not list(user_table.find({"username":username}))):
                user_table.insert({"username":username,"password":request.form['password']})
                follow_table.insert({"username":username,"follows":username})
                session['user']=username
                return redirect("/")
            else:
                return render_template("reg.html",error="User already exists")
@app.errorhandler(404)
def page_not_found(e):
    # note that we set the 404 status explicitly
    return render_template('404.html'), 404
if __name__ == '__main__':
    app.run(host="localhost",port=8088)    


 * Running on http://localhost:8088/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Nov/2019 12:01:34] "GET / HTTP/1.1" 302 -
127.0.0.1 - - [21/Nov/2019 12:01:34] "GET /home HTTP/1.1" 200 -


sidd


127.0.0.1 - - [21/Nov/2019 12:01:36] "GET /123 HTTP/1.1" 404 -
127.0.0.1 - - [21/Nov/2019 12:01:39] "GET / HTTP/1.1" 302 -
127.0.0.1 - - [21/Nov/2019 12:01:39] "GET /home HTTP/1.1" 200 -


sidd
